# Torpe Blockchain (तोर्पे ब्लोक्कचैन)
### A minimal blockchain data structure to understand the blockchain basics like hashing, nonce, states, genesis blocks etc.

![title](blockchain.jpg)

### Hash Function

A hash function is any function that can be used to map data of arbitrary size to data of a fixed size. The values returned by a hash function are called hash values, hash codes, digests, or simply hashes.

In [1]:
import hashlib
def hash_sha256(raw):
    raw = str(raw).encode('utf-8')
    return hashlib.sha256(raw).hexdigest() 

In [7]:
hash_sha256('Academic Activities British')

'0faac7a279d1b2a4c3a6ef2cf1a95b24224db99dc7336d70f8bbf260e8eee4d7'

In [8]:
hash_sha256('Academic Activities British')

'0faac7a279d1b2a4c3a6ef2cf1a95b24224db99dc7336d70f8bbf260e8eee4d7'

In [9]:
hash_sha256('Academic Activities BritisH')

'ad44e8eedad674dc10fef66ffde6884259e4947a60ead421204edcc0ca9b277a'

Hashes for the same text are the same as seen above.  Even when one character of the string is changed, the hash generated as a result seems to be completely random.  

SHA-256 collisions have not been found yet. 

### Nonce

> Number that can only be used once

An arbitrary number used in cryptography to ensure uniqueness and prevent the rerunning of transactions (known as replay attack). 

In [10]:
def hash_sha256_nonce(raw):
    raw_bytes = str(raw).encode('utf-8')
    hashed = hashlib.sha256(raw_bytes).hexdigest() 
    nonce = 0
    while (hashed[:5] != '00000'):
        nonce = nonce+1
        raw.update({'nonce': nonce})
        raw_bytes = str(raw).encode('utf-8')
        hashed = hashlib.sha256(raw_bytes).hexdigest() 
    return raw, hashed

In [12]:
print (hash_sha256_nonce({'text': 'secret', 'nonce': 0}))

({'text': 'secret', 'nonce': 1313721}, '0000071458e7cfbd2703c5384a7f5986cb0127f7a50bb7a92b7b10415a1c4999')


### Proof of work

The proof of work for this case will be to generate hashes with five leading zeros (by incrementing the nonce). This is the "mining" part.

## Block

Blocks hold batches of valid transactions that are hashed and encoded into a Merkle tree Each block includes the cryptographic hash of the prior block in the blockchain, linking the two. The linked blocks form a chain. This iterative process confirms the integrity of the previous block, all the way back to the original genesis block.

### Genesis Block

A genesis block or block0 is the first block of a block chain. The genesis block is almost always hardcoded into the software of the applications that utilize its block chain. It is a special case in that it does not reference a previous block

In [13]:
import datetime

In [53]:
# Lets assume 5 person were given 100 coins each
state = {
    'Person_1': 100, 
    'Person_2': 100, 
    'Person_3': 100, 
    'Person_4': 100, 
    'Person_5': 100
}

In [54]:
block0_data = {
    'timestamp': datetime.datetime.now(),
    'index': 0,
    'previous': None,
    'transactions': [state], 
    'nonce': 0
}

raw, hashed = hash_sha256_nonce(block0_data)
block0 = {
    'hash': hashed,
    'data': raw,
}

In [55]:
block0

{'hash': '0000073dec1d23825972d72f6aa3f0b76d3b38f0885f360fdcfb466fa5512d2b',
 'data': {'timestamp': datetime.datetime(2019, 12, 19, 16, 25, 50, 667043),
  'index': 0,
  'previous': None,
  'transactions': [{'Person_1': 100,
    'Person_2': 100,
    'Person_3': 100,
    'Person_4': 100,
    'Person_5': 100}],
  'nonce': 428344}}

This is the genesis block or block 0 here.

### Transactions
Lets create some random transactions. The transactions for the demo purpose follow +x, -x semantic. See the examples below.

In [56]:
import random
def random_transaction(state):
    temp_list = list(state.keys())
    random.shuffle(temp_list)
    # randomly select two persons
    first_person = temp_list.pop()
    second_person = temp_list.pop()
    receive = random.randint(1, 10)
    give = -receive
    return {
        first_person:receive, 
        second_person:give
    }

In [57]:
test_transactions = [random_transaction(state) for x in range(5)]

In [58]:
test_transactions

[{'Person_5': 4, 'Person_3': -4},
 {'Person_1': 7, 'Person_3': -7},
 {'Person_3': 5, 'Person_1': -5},
 {'Person_1': 8, 'Person_5': -8},
 {'Person_5': 10, 'Person_4': -10}]

### Updating State

In [59]:
def update_state(transaction, state):
    state = state.copy()
    for key in transaction:
        state[key] = state.get(key, 0) + transaction[key]
    return state

In [60]:
for transaction in test_transactions:
    state = update_state(transaction, state)

In [61]:
state

{'Person_1': 110,
 'Person_2': 100,
 'Person_3': 94,
 'Person_4': 90,
 'Person_5': 106}

### Valid Transactions

In [66]:
transaction.keys()
state.get('Person_5') +transaction.keys()

106

In [71]:
def check_transaction_validity(transaction, state):
    # check neg vs pos
    if sum(transaction.values()) is not 0:
        return False
    # check if amount in wallet to give
    for key in transaction.keys():
        if state.get(key, 0) + transaction[key] < 0:
            return False
    return True

In [72]:
for transaction in test_transactions:
    print (check_transaction_validity(transaction, state))

True
True
True
True
True


In [73]:
# No balance
print (check_transaction_validity({'A': 5, 'B': -5}, {'A': 0, 'B': 0}))

False


In [74]:
# Bad transaction
print (check_transaction_validity({'A': 5, 'B': 5}, {'A': 50, 'B': 50}))

False


### Initial State

In [75]:
# Let us reset
# Lets assume 5 person were given 100 coins each
state = {
    'Person_1': 100, 
    'Person_2': 100, 
    'Person_3': 100, 
    'Person_4': 100, 
    'Person_5': 100
}

blockchain = []

In [76]:
# Adding the genesis block
blockchain.append(block0)

In [77]:
blockchain

[{'hash': '0000073dec1d23825972d72f6aa3f0b76d3b38f0885f360fdcfb466fa5512d2b',
  'data': {'timestamp': datetime.datetime(2019, 12, 19, 16, 25, 50, 667043),
   'index': 0,
   'previous': None,
   'transactions': [{'Person_1': 100,
     'Person_2': 100,
     'Person_3': 100,
     'Person_4': 100,
     'Person_5': 100}],
   'nonce': 428344}}]

### Non-genesis block / New block

In [78]:
def new_block(transactions, blockchain):
    previous_block = blockchain[-1]
    data = {
        'timestamp': datetime.datetime.now(),
        'index': previous_block['data']['index'] + 1,
        'previous': previous_block['hash'],
        'transactions': transactions,
        'nonce': 0
        }
    
    raw, hashed = hash_sha256_nonce(data)
    block = {'hash': hashed, 'data': raw}
    return block

In [79]:
sample_transactions = [random_transaction(state) for x in range(50)]

In [80]:
sample_transactions

[{'Person_3': 10, 'Person_1': -10},
 {'Person_5': 10, 'Person_1': -10},
 {'Person_4': 8, 'Person_1': -8},
 {'Person_3': 6, 'Person_4': -6},
 {'Person_3': 2, 'Person_1': -2},
 {'Person_3': 7, 'Person_5': -7},
 {'Person_3': 2, 'Person_5': -2},
 {'Person_3': 10, 'Person_1': -10},
 {'Person_4': 3, 'Person_2': -3},
 {'Person_4': 4, 'Person_1': -4},
 {'Person_4': 5, 'Person_5': -5},
 {'Person_1': 9, 'Person_3': -9},
 {'Person_3': 3, 'Person_4': -3},
 {'Person_5': 6, 'Person_1': -6},
 {'Person_1': 2, 'Person_2': -2},
 {'Person_5': 1, 'Person_2': -1},
 {'Person_3': 9, 'Person_1': -9},
 {'Person_2': 3, 'Person_1': -3},
 {'Person_5': 9, 'Person_3': -9},
 {'Person_1': 10, 'Person_2': -10},
 {'Person_5': 2, 'Person_4': -2},
 {'Person_3': 8, 'Person_2': -8},
 {'Person_1': 5, 'Person_2': -5},
 {'Person_4': 10, 'Person_5': -10},
 {'Person_1': 9, 'Person_2': -9},
 {'Person_1': 8, 'Person_3': -8},
 {'Person_1': 8, 'Person_3': -8},
 {'Person_2': 8, 'Person_4': -8},
 {'Person_2': 7, 'Person_3': -7},
 {'P

### Transactions per block

Bitcoin blocks used to contain fewer than 200 transactions and the largest number of transactions in a block was 1,976 at in May 2013. In meanwhile (November 2017) the average number of transaction per block is well above 1500 with peaks above 2200.

In [81]:
# Assume block size is 5
transactions_per_block = 5
transaction_block = []

for transaction in sample_transactions:
    if check_transaction_validity(transaction, state):
        state = update_state(transaction, state)
        transaction_block.append(transaction)
        
        if len(transaction_block) >= transactions_per_block:
            blockchain.append(new_block(transaction_block, blockchain))
            transaction_block = []

In [82]:
import pprint
pp = pprint.PrettyPrinter()
for block in blockchain:
    pp.pprint(block)
    print('\n************************************************************************************\n')

{'data': {'index': 0,
          'nonce': 428344,
          'previous': None,
          'timestamp': datetime.datetime(2019, 12, 19, 16, 25, 50, 667043),
          'transactions': [{'Person_1': 100,
                            'Person_2': 100,
                            'Person_3': 100,
                            'Person_4': 100,
                            'Person_5': 100}]},
 'hash': '0000073dec1d23825972d72f6aa3f0b76d3b38f0885f360fdcfb466fa5512d2b'}

************************************************************************************

{'data': {'index': 1,
          'nonce': 791599,
          'previous': '0000073dec1d23825972d72f6aa3f0b76d3b38f0885f360fdcfb466fa5512d2b',
          'timestamp': datetime.datetime(2019, 12, 19, 16, 48, 45, 117776),
          'transactions': [{'Person_1': -10, 'Person_3': 10},
                           {'Person_1': -10, 'Person_5': 10},
                           {'Person_1': -8, 'Person_4': 8},
                           {'Person_3': 6, 'Person_4': -

### The current state
#### Syncing for the first time

In [83]:
def validate_block(block, parent, state):    
    error_msg = 'Error in %d' % block['data']['index']

    # check block hash
    assert block['hash'] == hash_sha256(block['data']), error_msg

    # check block indices
    assert block['data']['index'] == parent['data']['index'] + 1, error_msg

    # check previous hash
    assert block['data']['previous'] == parent['hash'], error_msg
    
    # validate all transactions
    for transaction in block['data']['transactions']:
        assert check_transaction_validity(transaction, state), error_msg
        state = update_state(transaction, state)
        
    return state

In [84]:
def check_chain(blockchain):
    state = {}

    for transaction in blockchain[0]['data']['transactions']:
        state = update_state(transaction, state)

    parent = blockchain[0]
    
    for block in blockchain[1:]:
        state = validate_block(block, parent, state)
        parent = block

    return state

check_chain(blockchain)

{'Person_1': 103,
 'Person_2': 81,
 'Person_3': 102,
 'Person_4': 93,
 'Person_5': 121}